In [2]:
import pickle

In [1]:
import pandas as pd

In [25]:
pip install tqdm

     |████████████████████████████████| 59 kB 7.8 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tqdm import tnrange, notebook,tqdm_notebook

In [3]:
infile = open('20200106_AISTripMainData.pkl','rb')
TripMain = pickle.load(infile)
infile.close()

In [4]:
infile = open('20200106_AISTripStatsData.pkl','rb')
TripStat = pickle.load(infile)
infile.close()

In [5]:
TripMain.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,Speed,point_id,Stationary,TimeCondition,medianSpeed,zScoreSpeed,Cos,trip_id
17394,636626,2017-07-31 16:52:06,37.16915,-76.60553,2.8,173.3,511.0,True,90.0,0.126114,5.044564,17394,True,1.323529,32.054958,-0.896133,0.000000,17277
17395,636626,2017-07-31 16:54:06,37.16767,-76.60873,5.3,-171.8,511.0,True,120.0,0.328257,9.847713,17395,False,1.764706,32.054958,-0.736777,0.999898,17277
17396,636626,2017-07-31 16:56:36,37.16584,-76.61284,5.3,-159.4,511.0,True,150.0,0.417732,10.025564,17396,False,2.205882,32.054958,-0.730877,0.955376,17277
17397,636626,2017-07-31 16:58:15,37.16548,-76.61585,5.4,-139.5,511.0,True,99.0,0.270312,9.829515,17397,False,1.455882,32.054958,-0.737381,0.998304,17277
17398,636626,2017-07-31 16:59:26,37.16534,-76.61815,6.4,-144.0,511.0,True,71.0,0.204873,10.387903,17398,False,1.044118,32.054958,-0.718855,0.998649,17277


In [6]:
TripStat.head()

,trip_id,MMSI,Dist,TimeDiff,Count,Speed,Cos
0,17277,636626,17.825440,1692.0,23,37.926467,0.889481
1,18037,641114,5.269649,1785.0,23,10.627863,0.905487
2,18220,641114,28.783308,9488.0,142,10.921154,0.940368
3,18222,641114,145.076523,38449.0,550,13.583591,0.978832
4,18502,641114,10.048207,3120.0,44,11.594085,0.942711


In [7]:
TripStat.columns

Index(['trip_id', 'MMSI', 'Dist', 'TimeDiff', 'Count', 'Speed', 'Cos'], dtype='object')

###  select trips in trip dataset

In [9]:
output = TripMain[TripMain['trip_id'].isin(TripStat['trip_id'].unique())]
output.to_csv('merged.csv',index=False)

In [37]:
# output = TripMain.merge(TripStat[['trip_id']],on=['trip_id'],how='left')
# output.to_csv('merged.csv',index=False)

In [3]:
output = pd.read_csv('merged.csv',parse_dates=['BaseDateTime'])
output = output.dropna(subset=['BaseDateTime','LAT','LON','trip_id'])

In [6]:

def sampleLat(df,OriginIndex,OriginLAT,OriginLON,TimeCount):
    if df['inserted'] != False:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lat = ((Lat1 - Lat0) / timeDelta*timeGap) + Lat0
            return lat
                
        else:
            return df['LAT']
    else:
        return df['LAT']
def sampleLon(df,OriginIndex,OriginLAT,OriginLON,TimeCount):
    if df['inserted'] != False:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lon = (Lon1 - Lon0) / timeDelta*timeGap + Lon0
#             print(location)

            return lon
        else:
            return df['LON']
    else:
        return df['LON']

In [30]:
output.columns

Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'mmsiDiff', 'TimeDiff', 'Dist', 'Speed', 'point_id', 'Stationary',
       'TimeCondition', 'medianSpeed', 'zScoreSpeed', 'Cos', 'trip_id'],
      dtype='object')

In [31]:
rawData = output[['MMSI', 'BaseDateTime', 'LAT', 'LON', 'trip_id']]

In [41]:
trip = output['trip_id'].unique().tolist()[0]
# _ = [(lambda x: x+1)(i) for i in notebook.tqdm(outputut['trip_id'].unique().tolist()[1:])]

outputSample = rawData[rawData['trip_id']==trip]
outputSample = outputSample.dropna()
outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                      - outputSample.iloc[0,1]).seconds)
count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
#     TimeDelta['inserted'] = True
TimeDelta['timeCount'] = [60*i for i in range(count+1)]
outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
outputSample['inserted'] = False
outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
outputSample = outputSample.reset_index()
del outputSample['index']
outputSample['Index'] = outputSample.index
outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
outputSample.loc[outputSample['inserted']==False,'timeCount_y'] = 0
outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
del outputSample['timeCount_y']
del outputSample['timeCount_x']
#     del outputSample['inserted_x']
#     del outputSample['inserted_y']
OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
# OriginIndex = [0] + OriginIndex
OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
# OriginLAT = [outputSample.iloc[0,2]] + OriginLAT
OriginLON = outputSample[outputSample['inserted']==False]['LON'].tolist()
# OriginLON = [outputSample.iloc[0,3]] + OriginLON
TimeCount = outputSample[outputSample['inserted']==False]['timeCount'].tolist()
# TimeCount = [outputSample.iloc[0,-1]] + TimeCount
outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
MMSI = outputSample.iloc[0,0]
tripID = outputSample['trip_id'].unique().tolist()[0]
# print(tripID)
outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)
outputSample['trip_id'] = outputSample['trip_id'].fillna(tripID)
f = open("doneTrip.csv", "w")
f.write('trip'+'\n')
f.write(str(outputSample.trip_id.unique().tolist()[0])+'\n')
f.close()
result = outputSample[outputSample['inserted']!=False]
result = result[['MMSI','trip_id','BaseDateTime','LAT','LON']]
result.to_csv('interpolated.csv',index=False)

In [22]:
doneTrip = pd.read_csv('doneTrip.csv')
doneTrip.head()

,trip
0,17277.0


In [ ]:
pbar = notebook.tqdm(total=len(output['trip_id'].unique().tolist()[1:]))
for trip in output['trip_id'].unique().tolist()[1:]:
    pbar.update(1)
    if trip in doneTrip.trip.unique().tolist():
        pass
    else:
        outputSample = rawData[rawData['trip_id']==trip]
        outputSample = outputSample.dropna()
        outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                              - outputSample.iloc[0,1]).seconds)
        count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
        timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
        TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
    #     TimeDelta['inserted'] = True
        TimeDelta['timeCount'] = [60*i for i in range(count+1)]
        outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
        outputSample['inserted'] = False
        outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
        outputSample = outputSample.reset_index()
        del outputSample['index']
        outputSample['Index'] = outputSample.index
        outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
        outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
        outputSample.loc[outputSample['inserted']==False,'timeCount_y'] = 0
        outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
    #     outputSample['inserted'] = outputSample['inserted_x'] * outputSample['inserted_y']
        del outputSample['timeCount_y']
        del outputSample['timeCount_x']
    #     del outputSample['inserted_x']
    #     del outputSample['inserted_y']
        OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
    #     OriginIndex = [0] + OriginIndex
        OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
    #     OriginLAT = [outputSample.iloc[0,2]] + OriginLAT
        OriginLON = outputSample[outputSample['inserted']!=True]['LON'].tolist()
    #     OriginLON = [outputSample.iloc[0,3]] + OriginLON
        TimeCount = outputSample[outputSample['inserted']!=True]['timeCount'].tolist()
    #     TimeCount = [outputSample.iloc[0,-1]] + TimeCount
        outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
        outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
        MMSI = outputSample.iloc[0,0]
        tripID = outputSample['trip_id'].unique().tolist()[0]
        f = open("doneTrip.csv", "a")
#         f.write('trip'+'\n')
        f.write(str(outputSample.trip_id.unique().tolist()[0])+'\n')
        f.close()
        outputSample = outputSample[outputSample['inserted']!=False]
        outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)
        
        outputSample['trip_id'] = outputSample['trip_id'].fillna(tripID)
        outputSample = outputSample[outputSample['inserted']!=False]
        result = outputSample[['MMSI','trip_id','BaseDateTime','LAT','LON']]
        result.to_csv('interpolated.csv',header=False,mode='a+',index=False)

In [189]:
trip = 9218801
# _ = [(lambda x: x+1)(i) for i in notebook.tqdm(outputut['trip_id'].unique().tolist()[1:])]

outputSample = output[output['trip_id']==trip]
outputSample = outputSample.dropna()
outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                      - outputSample.iloc[0,1]).seconds)
count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
#     TimeDelta['inserted'] = True
TimeDelta['timeCount'] = [60*i for i in range(count+1)]
outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
outputSample['inserted'] = False
# outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
# outputSample = outputSample.reset_index()
# del outputSample['index']
# outputSample['Index'] = outputSample.index
# outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
# outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
# outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
# #     outputSample['inserted'] = outputSample['inserted_x'] * outputSample['inserted_y']
# del outputSample['timeCount_y']
# del outputSample['timeCount_x']
# #     del outputSample['inserted_x']
# #     del outputSample['inserted_y']
# OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
# # OriginIndex = [0] + OriginIndex
# OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
# # OriginLAT = [outputSample.iloc[0,2]] + OriginLAT
# OriginLON = outputSample[outputSample['inserted']==False]['LON'].tolist()
# # OriginLON = [outputSample.iloc[0,3]] + OriginLON
# TimeCount = outputSample[outputSample['inserted']==False]['timeCount'].tolist()
# # TimeCount = [outputSample.iloc[0,-1]] + TimeCount
# outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
# outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
# MMSI = outputSample.iloc[0,0]
# outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)


In [194]:
outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])

In [197]:
outputSample.loc[outputSample['inserted']==False,'timeCount_y'] = 0

In [198]:
outputSample

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,...,point_id,Stationary,TimeCondition,medianSpeed,zScoreSpeed,Cos,trip_id,timeCount_x,inserted,timeCount_y
0,366968154.0,2017-07-13 23:59:01,40.43475,-74.08159,8.0,-80.4,511.0,True,90.0,0.212387,...,13661089.0,False,1.363636,26.545888,-1.883602,0.745109,9218801.0,0.0,False,0.0
1,366968154.0,2017-07-14 00:00:01,40.43713,-74.08203,10.1,13.2,511.0,True,60.0,0.266906,...,13661090.0,False,0.909091,26.545888,-1.098987,0.834667,9218801.0,60.0,False,0.0
39,NaN,2017-07-14 00:01:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0
2,366968154.0,2017-07-14 00:01:02,40.44013,-74.08076,11.6,18.2,511.0,True,61.0,0.350123,...,13661091.0,False,0.924242,26.545888,-0.613896,0.999858,9218801.0,121.0,False,0.0
40,NaN,2017-07-14 00:02:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,366968154.0,2017-07-14 00:40:09,40.49658,-73.93245,10.8,111.1,511.0,True,70.0,0.417879,...,13661125.0,False,1.060606,26.545888,-0.527497,0.999930,9218801.0,2468.0,False,0.0
78,NaN,2017-07-14 00:41:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2520.0
37,366968154.0,2017-07-14 00:41:19,40.49514,-73.92818,10.7,112.7,511.0,True,70.0,0.395726,...,13661126.0,False,1.060606,26.545888,-0.646385,0.999859,9218801.0,2538.0,False,0.0
79,NaN,2017-07-14 00:42:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2580.0


In [142]:
trip = 366968154.0
# _ = [(lambda x: x+1)(i) for i in notebook.tqdm(outputut['trip_id'].unique().tolist()[1:])]

outputSample = output[output['MMSI']==trip]
outputSample = outputSample.dropna()
outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                      - outputSample.iloc[0,1]).seconds)
count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
#     TimeDelta['inserted'] = True
TimeDelta['timeCount'] = [60*i for i in range(count+1)]
outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
outputSample['inserted'] = False
outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])
outputSample = outputSample.reset_index()
del outputSample['index']
outputSample['Index'] = outputSample.index
outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
#     outputSample['inserted'] = outputSample['inserted_x'] * outputSample['inserted_y']
del outputSample['timeCount_y']
del outputSample['timeCount_x']
#     del outputSample['inserted_x']
#     del outputSample['inserted_y']
OriginIndex = outputSample[outputSample['inserted']==False]['Index'].tolist()
# OriginIndex = [0] + OriginIndex
OriginLAT = outputSample[outputSample['inserted']==False]['LAT'].tolist()
# OriginLAT = [outputSample.iloc[0,2]] + OriginLAT
OriginLON = outputSample[outputSample['inserted']==False]['LON'].tolist()
# OriginLON = [outputSample.iloc[0,3]] + OriginLON
TimeCount = outputSample[outputSample['inserted']==False]['timeCount'].tolist()
# TimeCount = [outputSample.iloc[0,-1]] + TimeCount
outputSample['LAT'] = outputSample.apply(sampleLat, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
outputSample['LON'] = outputSample.apply(sampleLon, axis=1,args=(OriginIndex,OriginLAT,OriginLON,TimeCount))
MMSI = outputSample.iloc[0,0]
outputSample['MMSI'] = outputSample['MMSI'].fillna(MMSI)
outputSample = outputSample[outputSample['inserted']!=False]
outputSample = outputSample[['MMSI','BaseDateTime','LAT','LON']]
# outputSample.to_csv('interpolated.csv',index=False)

In [143]:
outputSample

,MMSI,BaseDateTime,LAT,LON
1,366968154.0,2017-07-01 15:18:51,40.432029,-74.082306
3,366968154.0,2017-07-01 15:19:51,40.432739,-74.080874
5,366968154.0,2017-07-01 15:20:51,40.433682,-74.080474
7,366968154.0,2017-07-01 15:21:51,40.435240,-74.081266
9,366968154.0,2017-07-01 15:22:51,40.437623,-74.081787
...,...,...,...,...
1525,366968154.0,2017-07-02 14:59:51,40.438973,-74.081297
1526,366968154.0,2017-07-02 15:00:51,40.438978,-74.081296
1527,366968154.0,2017-07-02 15:01:51,40.438983,-74.081296
1528,366968154.0,2017-07-02 15:02:51,40.438988,-74.081295


In [144]:
len(output.MMSI.unique())

5418

In [145]:
len(output.trip_id.unique())

81223

In [127]:
outputSample.iloc[0,0]

210464000

In [4]:
outputSample = output[output['trip_id']==17277]
outputSample = outputSample.dropna()
outputSample.head(2)

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,...,point_id,Stationary,TimeCondition,medianSpeed,zScoreSpeed,Cos,trip_id,LAT0,trip_id0,BaseDateTime0
1,636626,2017-07-31 16:54:06,37.16767,-76.60553,5.3,-171.8,511.0,True,120.0,0.328257,...,17395,False,1.764706,32.054958,-0.736777,0.999898,17277,37.16915,17277.0,2017-07-31 16:52:06
2,636626,2017-07-31 16:56:36,37.16584,-76.60873,5.3,-159.4,511.0,True,150.0,0.417732,...,17396,False,2.205882,32.054958,-0.730877,0.955376,17277,37.16767,17277.0,2017-07-31 16:54:06


In [5]:
outputSample['timeCount'] = outputSample['BaseDateTime'].apply(lambda x: (pd.Timestamp(x)
                                                                          - outputSample.iloc[0,1]).seconds)
# outputSample.head()
# (pd.Timestamp(outputSample['BaseDateTime']) - outputSample.iloc[0,1]).seconds

In [6]:
count = (outputSample.iloc[-1,1] - outputSample.iloc[0,1]).seconds // 60
timeDelta = [str((outputSample.iloc[0,1] + pd.Timedelta(seconds=60*i))) for i in range(count+1)]
TimeDelta = pd.DataFrame({'BaseDateTime':timeDelta})
TimeDelta['inserted'] = True
TimeDelta['timeCount'] = [60*i for i in range(count+1)]

In [7]:
outputSample['BaseDateTime'] = outputSample['BaseDateTime'].astype('str')
outputSample = outputSample.merge(TimeDelta,on=['BaseDateTime'],how='outer').sort_values(by=['BaseDateTime'])

In [8]:
outputSample = outputSample.reset_index()
del outputSample['index']
outputSample['Index'] = outputSample.index

outputSample.head(3)

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,...,zScoreSpeed,Cos,trip_id,LAT0,trip_id0,BaseDateTime0,timeCount_x,inserted,timeCount_y,Index
0,636626.0,2017-07-31 16:54:06,37.16767,-76.60553,5.3,-171.8,511.0,True,120.0,0.328257,...,-0.736777,0.999898,17277.0,37.16915,17277.0,2017-07-31 16:52:06,0.0,True,0.0,0
1,NaN,2017-07-31 16:55:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,True,60.0,1
2,NaN,2017-07-31 16:56:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,True,120.0,2


In [9]:
outputSample['timeCount_x'] = outputSample['timeCount_x'].fillna(0)
outputSample['timeCount_y'] = outputSample['timeCount_y'].fillna(0)
outputSample['timeCount'] = outputSample['timeCount_y'] + outputSample['timeCount_x']
del outputSample['timeCount_y']
del outputSample['timeCount_x']

In [10]:
outputSample.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,...,medianSpeed,zScoreSpeed,Cos,trip_id,LAT0,trip_id0,BaseDateTime0,inserted,Index,timeCount
0,636626.0,2017-07-31 16:54:06,37.16767,-76.60553,5.3,-171.8,511.0,True,120.0,0.328257,...,32.054958,-0.736777,0.999898,17277.0,37.16915,17277.0,2017-07-31 16:52:06,True,0,0.0
1,NaN,2017-07-31 16:55:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,1,60.0
2,NaN,2017-07-31 16:56:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,2,120.0
3,636626.0,2017-07-31 16:56:36,37.16584,-76.60873,5.3,-159.4,511.0,True,150.0,0.417732,...,32.054958,-0.730877,0.955376,17277.0,37.16767,17277.0,2017-07-31 16:54:06,NaN,3,150.0
4,NaN,2017-07-31 16:57:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,True,4,180.0


In [11]:
OriginIndex = outputSample[outputSample['inserted']!=True]['Index'].tolist()
OriginIndex = [0] + OriginIndex
OriginLAT = outputSample[outputSample['inserted']!=True]['LAT'].tolist()
OriginLAT = [outputSample.iloc[0,2]] + OriginLAT
OriginLON = outputSample[outputSample['inserted']!=True]['LON'].tolist()
OriginLON = [outputSample.iloc[0,3]] + OriginLON
TimeCount = outputSample[outputSample['inserted']!=True]['timeCount'].tolist()
TimeCount = [outputSample.iloc[0,-1]] + TimeCount

In [39]:

def sampleLat(df):
    if df['inserted'] == True:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lat = ((Lat1 - Lat0) / timeDelta*timeGap) + Lat0
#             if insertedIndex == 47:
#                 print(Lat1,Lat0,timeDelta,timeGap,lat)
            return lat
        else:
            return df['LAT']
    else:
        return df['LAT']
def sampleLon(df):
    if df['inserted'] == True:
        insertedIndex = df['Index']
        if insertedIndex > 0:
            wholeIndex = OriginIndex + [insertedIndex]
            wholeIndex.sort()
            location = wholeIndex.index(insertedIndex)
#             print(location)
            Lat0 = OriginLAT[location-1]
            Lon0 = OriginLON[location-1]
            Lat1 = OriginLAT[location]
            Lon1 = OriginLON[location]
            timeDelta = TimeCount[location] - TimeCount[location-1] 
            currentTime = df['timeCount']
            timeGap = currentTime - TimeCount[location-1]
            lon = (Lon1 - Lon0) / timeDelta*timeGap + Lon0

            return lon
        else:
            return df['LON']
    else:
        return df['LON']

In [40]:
outputSample['LAT'] = outputSample.apply(sampleLat, axis=1)
outputSample['LON'] = outputSample.apply(sampleLon, axis=1)

37.05277 37.05315 68.0 56.0 37.052837058823535


In [42]:
outputSample.to_csv('OneTripSample.csv',index=False)

In [41]:
outputSample

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,mmsiDiff,TimeDiff,Dist,...,Cos,trip_id,LAT0,trip_id0,BaseDateTime0,inserted,Index,timeCount,LAT_,LON_
0,636626.0,2017-07-31 16:54:06,37.167670,-76.605530,5.3,-171.8,511.0,True,120.0,0.328257,...,0.999898,17277.0,37.16915,17277.0,2017-07-31 16:52:06,True,0,0.0,37.167670,-76.605530
1,NaN,2017-07-31 16:55:06,37.166938,-76.606810,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,1,60.0,37.166938,-76.606810
2,NaN,2017-07-31 16:56:06,37.166206,-76.608090,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,2,120.0,37.166206,-76.608090
3,636626.0,2017-07-31 16:56:36,37.165840,-76.608730,5.3,-159.4,511.0,True,150.0,0.417732,...,0.955376,17277.0,37.16767,17277.0,2017-07-31 16:54:06,NaN,3,150.0,37.165840,-76.608730
4,NaN,2017-07-31 16:57:06,37.165731,-76.609975,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,4,180.0,37.165185,-76.609975
5,NaN,2017-07-31 16:58:06,37.165513,-76.612466,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,5,240.0,37.164967,-76.612466
6,636626.0,2017-07-31 16:58:15,37.165480,-76.612840,5.4,-139.5,511.0,True,99.0,0.270312,...,0.998304,17277.0,37.16584,17277.0,2017-07-31 16:56:36,NaN,6,249.0,37.165480,-76.612840
7,NaN,2017-07-31 16:59:06,37.165379,-76.615002,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,7,300.0,37.164888,-76.615002
8,636626.0,2017-07-31 16:59:26,37.165340,-76.615850,6.4,-144.0,511.0,True,71.0,0.204873,...,0.998649,17277.0,37.16548,17277.0,2017-07-31 16:58:15,NaN,8,320.0,37.165340,-76.615850
9,NaN,2017-07-31 17:00:06,37.165327,-76.617358,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,True,9,360.0,37.165222,-76.617358
